In [1]:
!pip install sastrawi

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string
import nltk

from io import StringIO
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

seed = 0
np.random.seed(seed)

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
app_reviews_df = pd.read_csv('review_gojek.csv')

app_reviews_df.to_csv('review_gojek.csv', index=False)

jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

In [4]:
app_reviews_df.head()

,Review
0,"Aplikasi sudah sangat baik, tapi saya kecewa, ..."
1,Ga tau kenapa sering kali pihak driver (gocar)...
2,"Opsi pemilihan GOFOOD ""terdekat"" sekarang kura..."
3,"Cuma mau kasih tau aja, itu iklan Go-Jek yg di..."
4,"Saya pengguna Gopayletter, saya sudah membayar..."


In [5]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216000 entries, 0 to 215999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Review  215999 non-null  object
dtypes: object(1)
memory usage: 1.6+ MB


In [6]:
app_reviews_df.isnull().any()

Review    True
dtype: bool

In [7]:
clean_df = app_reviews_df.dropna()
clean_df.isnull().any()

Review    False
dtype: bool

In [8]:
clean_df = clean_df.drop_duplicates()
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 209516 entries, 0 to 215999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Review  209516 non-null  object
dtypes: object(1)
memory usage: 3.2+ MB


In [9]:
def cleaningText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text)
  text = re.sub(r'#[A-Za-z0-9]+', '', text)
  text = re.sub(r'RT[\s]', '', text)
  text = re.sub(r'[0-9]+', '', text)
  text = re.sub(r'[^\w\s]', '', text)
  text = text.replace('\n', '')
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = text.strip(' ')
  return text

def casefoldingText(text):
  text = text.lower()
  return text

def tokenizingText(text):
  text = word_tokenize(text)
  return text

def filteringText(text):
  factory = StopWordRemoverFactory()
  stopwords_sastrawi = factory.get_stop_words()
  listStopwords = set(stopwords_sastrawi)
  listStopwords_En = set(stopwords.words('english'))
  listStopwords.update(listStopwords_En)

  listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy", 'hey', 'heyy', 'haloo', 'halo', 'hii', 'hi'])
  filtered = []
  for txt in text:
    if txt not in listStopwords:
      filtered.append(txt)
  text = filtered
  return text

def stemmingText(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  words = text.split()
  stemmed_words = [stemmer.stem(word) for word in words]
  stemmed_text = ' '.join(stemmed_words)

  return stemmed_text

def toSentence(list_words):
  sentence = ' '.join(word for word in list_words)
  return sentence

In [10]:
slangwords = {"gpp": "nggak apa-apa",
    "tdk": "tidak",
    "tq": "terima kasih",
    "makasi": "terima kasih",
    "mksh": "terima kasih",
    "trm": "terima",
    "ok": "oke",
    "okey": "oke",
    "udh": "sudah",
    "sm": "sama",
    "dri": "dari",
    "kmn": "kemana",
    "krn": "karena",
    "msh": "masih",
    "dkt": "dekat",
    "bs": "bisa",
    "gak": "nggak",
    "nggk": "nggak",
    "pls": "tolong",
    "jg": "juga",
    "org": "orang",
    "dr": "dari",
    "km": "kamu",
    "knp": "kenapa",
    "blg": "bilang",
    "kyk": "kayak",
    "sgt": "sangat",
    "dlm": "dalam",
    "utk": "untuk",
    "smg": "semoga",
    "lbh": "lebih",
    "jl": "jalan",
    "jln": "jalan",
    "lg": "lagi",
    "sblm": "sebelum",
    "dngn": "dengan",
    "td": "tadi",
    "skrg": "sekarang",
    "sy": "saya",
    "gw": "gue",
    "ane": "saya",
    "lo": "kamu",
    "lu": "kamu",
    "mau": "ingin",
    "plg": "pulang",
    "nyampe": "sampai",
    "sampenya": "sampainya",
    "blm": "belum",
    "nganter": "mengantar",
    "nganterin": "mengantar",
    "nganternya": "mengantarnya",
    "dpt": "dapat",
    "dapet": "dapat",
    "lelet": "lambat",
    "lambrett": "lambat",
    "lama": "lambat",
    "cepat": "cepat",
    "ngebut": "cepat",
    "pelayanannya": "pelayanannya",
    "stgh": "setengah",
    "bnyk": "banyak",
    "brp": "berapa",
    "dpt": "dapat",
    "dapet": "dapat",
    "ga": "nggak",
    "tp": "tapi",
    "brngkt": "berangkat",
    "nyampe": "sampai",
    "kyk": "seperti",
    "beres": "selesai",
    "jkt": "Jakarta",
    "bgr": "Bogor",
    "bdg": "Bandung",
    "sby": "Surabaya",
    "mlg": "Malang",
    "smg": "Semarang",
    "sjk": "sejak",
    "smp": "sampai",
    "rma": "rumah",
    "mntn": "maintain",
    "mntap": "mantap",
    "nyaman": "nyaman",
    "tmpt": "tempat",
    "tmn": "teman",
    "kl": "kalau",
    "klau": "kalau",
    "ak": "aku",
    "mnrt": "menurut",
    "mnrtku": "menurutku",
    "bgini": "begini",
    "bgitu": "begitu",
    "tmptnya": "tempatnya",
    "brangkat": "berangkat",
    "prgi": "pergi",
    "balik": "pulang",
    "pas": "ketika",
    "plng": "pulang",
    "jgn": "jangan",
    "jd": "jadi",
    "dmn": "dimana",
    "pd": "pada",
    "skli": "sekali",
    "brng": "barang",
    "sbrng": "seberang",
    "bngt": "banget",
    "btw": "by the way",
    "btp": "betapa",
    "kbnykan": "kebanyakan",
    "kcl": "kecil",
    "bsr": "besar",
    "krng": "kurang",
    "kn": "kan",
    "br": "baru",
    "jdul": "judul",
    "pndpt": "pendapat",
    "plhn": "pilihan",
    "thx": "thanks",
    "ty": "thank you",
    "btw": "by the way",
    "tnpa": "tanpa",
    "tpi": "tapi",
    "hrus": "harus",
    "nnti": "nanti",
    "pkrn": "pikiran",
    "byk": "banyak",
    "slmt": "selamat",
    "wrga": "warga",
    "wrg": "warga",
    "bkin": "bikin",
    "lgi": "lagi",
    "gampang": "mudah",
    "susah": "sulit",
    "temen": "teman",
    "kren": "keren",
    "brgnya": "barangnya",
    "jamnya": "jamnya",
    "tgl": "tanggal",
    "tglnya": "tanggalnya",
    "brngkt": "berangkat",
    "brng": "barang",
    "bntr": "sebentar",
    "br": "baru",
    "drmn": "dari mana",
    "drng": "darang",
    "nnti": "nanti",
    "mncr": "mancar",
    "brrti": "berarti",
    "cmn": "cuman",
    "knp": "kenapa",
    "spya": "supaya",
    "gmn": "gimana",
    "knp": "kenapa",
    "gmna": "gimana",
    "blng": "bilang",
    "bpk": "bapak",
    "smua": "semua",
    "smwa": "semua",
    "syg": "sayang",
    "pak": "bapak",
    "trus": "terus",
    "klwrga": "keluarga",
    "dlm": "dalam",
    "smoga": "semoga",
    "ngk": "nggak",
    "tdi": "tadi",
    "tdk": "tidak",
    "bljr": "belajar",
    "plgn": "pelanggan",
    "spt": "seperti",
    "tsb": "tersebut",
    "tgl": "tanggal",
    "klo": "kalau",
    "saat2": "saat-saat",
    "waktu2": "waktu-waktu",
    "sy": "saya",
    "kdg": "kadang",
    "slalu": "selalu",
    "nganter": "mengantar",
    "mpls": "empilis",
    "ntar": "nanti",
    "td": "tadi",
    "kmr": "kemarin",
    "gampang": "mudah",
    "mntp": "mantap",
    "tmn": "teman",
    "bgmn": "bagaimana",
    "msh": "masih",
    "ksih": "kasih",
    "nnti": "nanti",
    "hr": "hari",
    "mna": "mana",
    "bsok": "besok",
    "dng": "dengan",
    "pngn": "pengen",
    "dlu": "dulu",
    "sni": "sini",
    "sna": "sana",
    "ll": "lalu",
    "krng": "kurang",
    "trs": "terus",
    "tk": "terima kasih",
    "tq": "terima kasih",
    "brngkt": "berangkat",
    "lmbt": "lambat",
    "plng": "pulang",
    "dri": "dari",
    "jkt": "Jakarta",
    "bdg": "Bandung",
    "smg": "Semarang",
    "sby": "Surabaya",
    "mks": "Makassar",
    "mlg": "Malang",
    "dkt": "dekat",
    "pjg": "panjang",
    "prgi": "pergi",
    "smpai": "sampai",
    "cpt": "cepat",
    "kek": "seperti",
    "kayak": "seperti",
    "jau": "jauh",
    "ngomong": "berbicara",
    "ngomel": "mengomel",
    "trmksih": "terima kasih",
    "mksih": "terima kasih",
    "kasian": "kasihan",
    "ksian": "kasihan",
    "liat": "melihat",
    "kliatan": "kelihatan",
    "lngsg": "langsung",
    "gk": "nggak",
    "enggak": "tidak",
    "tmpt": "tempat",
    "t4": "tempat",
    "pindah": "berpindah",
    "btul": "betul",
    "btul2": "betul-betul",
    "trs": "terus",
    "sm2": "sama-sama",
    "drmn": "dari mana",
    "drpd": "daripada",
    "ngga": "tidak",
    "kt": "kita",
    "drmn": "darimana",
    "dpn": "depan",
    "blkg": "belakang",
    "atas": "di atas",
    "bwh": "bawah",
    "lngsng": "langsung",
    "kn": "kan",
    "syg": "sayang",
    "smg": "semoga",
    "ngak": "tidak",
    "ntr": "nanti",
    "ktm": "ketemu",
    "ngebut": "cepat",
    "jl": "jalan",
    "jln": "jalan",
    "trus": "terus",
    "brgkt": "berangkat",
    "blm": "belum",
    "bntar": "sebentar",
    "cmn": "cuma",
    "mlm": "malam",
    "pg": "pagi",
    "tdi": "tadi",
    "bs": "bisa",
    "gpp": "tidak apa-apa",
    "kmn": "kemana",
    "btw": "omong-omong",
    "n": "dan",
    "u": "untuk",
    "d": "di",
    "g": "tidak",
    "u/": "untuk",
    "dgn": "dengan",
    "dulu": "sebelumnya",
    "jgn": "jangan",
    "smgt": "semangat",
    "dmn": "di mana",
    "temen": "teman",
    "aq": "aku",
    "tdk": "tidak",
    "btw": "omong-omong",
    "kmrn": "kemarin",
    "dl": "dulu",
    "kpn": "kapan",
    "smua": "semua",
    "bgs": "bagus",
    "bnyk": "banyak",
    "dtg": "datang",
    "gtu": "begitu",
    "mlm": "malam",
    "pngn": "ingin",
    "gak": "tidak",
    "ya": "ya",
    "yaudah": "ya sudah",
    "yaudah": "ya sudah",
    "yaudah": "ya"
    }

In [11]:
def fix_slangwords(text):
  words = text.split()
  fixed_words = []

  for word in words:
    if word.lower() in slangwords:
      fixed_words.append(slangwords[word.lower()])
    else:
      fixed_words.append(word)

  fixed_text = ' '.join(fixed_words)
  return fixed_text

In [12]:
clean_df['content_clean'] = clean_df['Review'].apply(cleaningText)
clean_df['content_casefolding'] = clean_df['content_clean'].apply(casefoldingText)
clean_df['content_slangwords'] = clean_df['content_casefolding'].apply(fix_slangwords)
clean_df['content_tokenizing'] = clean_df['content_slangwords'].apply(tokenizingText)
clean_df['content_stopwords'] = clean_df['content_tokenizing'].apply(filteringText)
clean_df['content_akhir'] = clean_df['content_stopwords'].apply(toSentence)

In [13]:
clean_df

,Review,content_clean,content_casefolding,content_slangwords,content_tokenizing,content_stopwords,content_akhir
0,"Aplikasi sudah sangat baik, tapi saya kecewa, ...",Aplikasi sudah sangat baik tapi saya kecewa ha...,aplikasi sudah sangat baik tapi saya kecewa ha...,aplikasi sudah sangat baik tapi saya kecewa ha...,"[aplikasi, sudah, sangat, baik, tapi, saya, ke...","[aplikasi, sangat, baik, kecewa, harusnya, pih...",aplikasi sangat baik kecewa harusnya pihak goj...
1,Ga tau kenapa sering kali pihak driver (gocar)...,Ga tau kenapa sering kali pihak driver gocar c...,ga tau kenapa sering kali pihak driver gocar c...,nggak tau kenapa sering kali pihak driver goca...,"[nggak, tau, kenapa, sering, kali, pihak, driv...","[tau, sering, kali, pihak, driver, gocar, canc...",tau sering kali pihak driver gocar cancel sesu...
2,"Opsi pemilihan GOFOOD ""terdekat"" sekarang kura...",Opsi pemilihan GOFOOD terdekat sekarang kurang...,opsi pemilihan gofood terdekat sekarang kurang...,opsi pemilihan gofood terdekat sekarang kurang...,"[opsi, pemilihan, gofood, terdekat, sekarang, ...","[opsi, pemilihan, gofood, terdekat, sekarang, ...",opsi pemilihan gofood terdekat sekarang kurang...
3,"Cuma mau kasih tau aja, itu iklan Go-Jek yg di...",Cuma mau kasih tau aja itu iklan GoJek yg dike...,cuma mau kasih tau aja itu iklan gojek yg dike...,cuma ingin kasih tau aja itu iklan gojek yg di...,"[cuma, ingin, kasih, tau, aja, itu, iklan, goj...","[cuma, kasih, tau, aja, iklan, gojek, yg, dike...",cuma kasih tau aja iklan gojek yg dikelola pan...
4,"Saya pengguna Gopayletter, saya sudah membayar...",Saya pengguna Gopayletter saya sudah membayar ...,saya pengguna gopayletter saya sudah membayar ...,saya pengguna gopayletter saya sudah membayar ...,"[saya, pengguna, gopayletter, saya, sudah, mem...","[pengguna, gopayletter, membayar, tagihan, mem...",pengguna gopayletter membayar tagihan memang t...
...,...,...,...,...,...,...,...
215995,Apl ini tidak bagus saya foto KTP ngak bisa pa...,Apl ini tidak bagus saya foto KTP ngak bisa pa...,apl ini tidak bagus saya foto ktp ngak bisa pa...,apl ini tidak bagus saya foto ktp tidak bisa p...,"[apl, ini, tidak, bagus, saya, foto, ktp, tida...","[apl, bagus, foto, ktp, parah, parah]",apl bagus foto ktp parah parah
215996,good.. sangt bermanfaat.,good sangt bermanfaat,good sangt bermanfaat,good sangt bermanfaat,"[good, sangt, bermanfaat]","[good, sangt, bermanfaat]",good sangt bermanfaat
215997,Mencari lokasi ngga bisa,Mencari lokasi ngga bisa,mencari lokasi ngga bisa,mencari lokasi tidak bisa,"[mencari, lokasi, tidak, bisa]","[mencari, lokasi]",mencari lokasi
215998,APP ENGGK JELAS DI TOTUR PAKE SIM BISA PAS DI ...,APP ENGGK JELAS DI TOTUR PAKE SIM BISA PAS DI ...,app enggk jelas di totur pake sim bisa pas di ...,app enggk jelas di totur pake sim bisa ketika ...,"[app, enggk, jelas, di, totur, pake, sim, bisa...","[app, enggk, jelas, totur, pake, sim, coba]",app enggk jelas totur pake sim coba


In [14]:
positive_lexicon = {
    "baik": 4,
    "bagus": 5,
    "cepat": 4,
    "ramah": 5,
    "tepat": 4,
    "nyaman": 5,
    "murah": 3,
    "puas": 5,
    "aman": 4,
    "bersih": 4,
    "terjangkau": 3,
    "rekomendasi": 5,
    "mantap": 4,
    "sopan": 5,
    "handal": 4,
    "akurat": 4,
    "hemat": 3,
    "praktis": 4,
    "efisien": 4,
    "memuaskan": 5,
    "responsif": 4,
    "komplit": 4,
    "terpercaya": 4,
    "profesional": 5,
    "top": 4,
    "hebat": 4,
    "oke": 3,
    "terbaik": 5,
    "enak": 4,
    "canggih": 4,
    "menyenangkan": 5,
    "berkelas": 4,
    "istimewa": 5,
    "favorit": 5,
    "solutif": 4,
    "luwes": 4,
    "seru": 4,
    "keren": 5,
    "super": 5,
    "kagum": 4,
    "memudahkan": 5,
    "menakjubkan": 5,
    "terbantu": 4,
    "terpuji": 4,
    "mengagumkan": 5,
    "mempermudah": 5,
    "kekinian": 4,
    "jempolan": 4,
    "worth": 4,
    "senang": 4,
    "inovatif": 5,
    "gemilang": 5,
    "cocok": 4,
    "mengesankan": 5,
    "menawan": 5,
    "perhatian": 4,
    "membantu": 5,
    "amazing": 5,
    "menguntungkan": 4,
    "diandalkan": 4,
    "simpel": 4,
    "mudah": 4,
    "terdepan": 4,
    "sempurna": 5,
    "happy": 4,
    "asyik": 4,
    "paten": 4,
    "mantep": 4,
    "membanggakan": 5,
    "teratur": 4,
    "apik": 4,
    "mengasyikkan": 4,
    "sip": 4,
    "berguna": 4,
    "wow": 5,
    "fantastis": 5,
    "superior": 5,
    "gampang": 4,
    "dasyat": 5,
    "maksimal": 4,
    "mewah": 4,
    "bermanfaat": 4,
    "solusi": 4
}

negative_lexicon = {
    "buruk": -3,
    "lambat": -2,
    "mengecewakan": -4,
    "mahal": -2,
    "kurang": -2,
    "kotor": -3,
    "parah": -4,
    "rusak": -3,
    "jahat": -5,
    "ancaman": -5,
    "terlambat": -3,
    "membahayakan": -5,
    "marah": -3,
    "emosi": -3,
    "kesal": -3,
    "benci": -5,
    "jelek": -2,
    "berbahaya": -4,
    "salah": -2,
    "penipu": -5,
    "curang": -4,
    "kecewa": -3,
    "lelet": -2,
    "sembarangan": -3,
    "mengerikan": -4,
    "malas": -3,
    "bohong": -4,
    "tipu": -4,
    "asal": -3,
    "brutal": -5,
    "musibah": -4,
    "cacat": -4,
    "payah": -3,
    "bubar": -3,
    "gagal": -4,
    "kejam": -5
}

In [15]:
def sentiment_analysis_lexicon_indonesia(text):
  score = 0
  for word in text:
    if(word in positive_lexicon):
      score += positive_lexicon[word]
  for word in text:
    if(word in negative_lexicon):
      score += negative_lexicon[word]

  polarity = ''

  if(score > 0):
    polarity = 'positive'
  elif(score < 0):
    polarity = 'negative'
  else:
    polarity = 'neutral'

  return score, polarity

In [16]:
results = clean_df['content_stopwords'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    87432
neutral     85936
negative    36148
Name: count, dtype: int64


In [17]:
X = clean_df['content_akhir']
y = clean_df['polarity']

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=1000, min_df=100, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
kfold = KFold(n_splits=5, random_state=42, shuffle=True)

In [20]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

lgbmmodel = lgb.LGBMClassifier()
lgbmmodel.fit(X_train.toarray(), y_train)

y_pred_train = lgbmmodel.predict(X_train.toarray())
y_pred_test = lgbmmodel.predict(X_test.toarray())

accuracy_train = accuracy_score(y_pred_train, y_train)
accuracy_test = accuracy_score(y_pred_test, y_test)

print(accuracy_train)
print(accuracy_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.099118 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 190553
[LightGBM] [Info] Number of data points in the train set: 167612, number of used features: 1000
[LightGBM] [Info] Start training from score -1.755287
[LightGBM] [Info] Start training from score -0.890506
[LightGBM] [Info] Start training from score -0.875402
0.9834498723241772
0.9813860252004581


In [21]:
from sklearn.ensemble import GradientBoostingClassifier

gbm_model = GradientBoostingClassifier()
gbm_model.fit(X_train.toarray(), y_train)

y_pred_train = gbm_model.predict(X_train.toarray())
y_pred_test = gbm_model.predict(X_test.toarray())

accuracy_train = accuracy_score(y_pred_train, y_train)
accuracy_test = accuracy_score(y_pred_test, y_test)

print(accuracy_train)
print(accuracy_test)

0.9469668042860893
0.9464728904161894


In [22]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
new_y = label_encoder.fit_transform(y)

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_tfidf, new_y)
xgb_accuracy = cross_val_score(xgb_model, X_tfidf, new_y, cv=kfold)

print(xgb_accuracy)
print(xgb_accuracy.mean())

[0.98157694 0.9811708  0.98145717 0.98045486 0.98145717]
0.9812233893662136


In [23]:
def preprocess_text(text):
  new_text_cleaned = cleaningText(text)
  new_text_casefolded = casefoldingText(new_text_cleaned)
  new_text_slangfixed = fix_slangwords(new_text_casefolded)
  new_text_tokenized = tokenizingText(new_text_slangfixed)
  new_text_filtered = filteringText(new_text_tokenized)
  new_text_final = toSentence(new_text_filtered)
  return new_text_final

def predict_sentiment(text, model):
  text_processed = preprocess_text(text)
  X_text = tfidf.transform([text_processed])
  prediction = model.predict(X_test.toarray())
  return prediction

kalimat_baru = input("Masukkan kalimat baru: ")

prediksi_lgb = predict_sentiment(kalimat_baru, lgbmmodel)
if prediksi_lgb[0] == 'positive':
    print("Sentimen LGBM adalah POSITIF.")
elif prediksi_lgb[0] == 'negative':
    print("Sentimen LGBM adalah NEGATIF.")
else:
    print("Sentimen LGBM adalah NEUTRAL.")

prediksi_gbm = predict_sentiment(kalimat_baru, gbm_model)
if prediksi_gbm[0] == 'positive':
    print("Sentimen GBM adalah POSITIF.")
elif prediksi_gbm[0] == 'negative':
    print("Sentimen GBM adalah NEGATIF.")
else:
    print("Sentimen GBM adalah NEUTRAL.")

prediksi_xgb = predict_sentiment(kalimat_baru, xgb_model)
prediksi_xgb_label = label_encoder.inverse_transform(prediksi_xgb)
if prediksi_xgb_label[0] == 'positive':
    print("Sentimen XGBoost adalah POSITIF.")
elif prediksi_xgb_label[0] == 'negative':
    print("Sentimen XGBoost adalah NEGATIF.")
else:
    print("Sentimen XGBoost adalah NEUTRAL.")

Masukkan kalimat baru: aplikasi ini sangat baik
Sentimen LGBM adalah POSITIF.
Sentimen GBM adalah POSITIF.
Sentimen XGBoost adalah POSITIF.
